In [60]:
import rasterio as rio
import numpy as np

In [61]:
data = r'W:\NLCDFS1\lcnext\Assessment_2023\data\landsat_level_3_products\DSWE\h16v04\2021\LC08_CU_016004_20210927_20211004_02_DIAG.tif'

In [62]:
with rio.open(data) as dswe:
    dswe_data = dswe.read(1)
    dswe.profile
    print(dswe.count)

1


In [63]:
values = np.unique(dswe_data)

In [6]:
out_data = {}

In [9]:
out_data = np.zeros_like(values, dtype=np.uint8)

In [10]:
# confident water
out_data[np.where((values == 1111) | (values == 10111) | (values == 11011) | (values == 11101) | (values == 11110) | (values == 11111))] = 1

out_data[np.where((values == 111) | (values == 1011) | (values == 1101) | (values == 1110) | (values == 10011) | (values == 10101) | (values == 10110) | (values == 11001) | (values == 11010) | (values == 11100))] = 2

out_data[np.where(values == 11000)] = 3

out_data[np.where((values == 11) | (values == 101) | (values == 101) | (values == 110) | (values == 1001) | (values == 1010) | (values == 1100) | (values == 10000) | (values == 10001) | (values == 10010) | (values == 10100))] = 4



In [11]:
out_data

array([0, 0, 0, 0, 0, 4, 4, 2, 0, 1, 4, 4, 4, 2, 4, 2, 2, 1, 3, 2, 2, 1,
       2, 1, 1, 1], dtype=uint8)

In [12]:
check = np.unique(out_data)

In [54]:
check.shape

(5,)

In [19]:
out_data = np.where(check == -9999, 0, check)

In [58]:
# shape array with 1 rows and5 columns
array2 = check.reshape(1, 5)

In [68]:
# Register GDAL format drivers and configuration options with a
# context manager.
with rio.Env():

    # Write an array as a raster band to a new 8-bit file. For
    # the new file's profile, we start with the profile of the source
    profile = dswe.profile
    

    # And then change the band count to 1, set the
    # dtype to uint8, and specify LZW compression.
    profile.update(
        dtype=rio.uint8,
        count=1,
        nodata=0,
        compress='lzw')

    with rio.open('example.tif', 'w', **profile) as dst:
        dst.write(array2.astype(rio.uint8), 1)

# At the end of the ``with rasterio.Env()`` block, context
# manager exits and all drivers are de-registered.